# 可视化
https://blog.csdn.net/weixin_42632271/article/details/107458445


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tensorboardX import SummaryWriter
from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #in 1* (28*28) => 10* (24*24)
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        #in 10 * ( 12*12 ) => 20* (8*8 )
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        # 
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        #  max_pool2d , 第一个参数输入，第二个参数 kernel_size
        #  in  10* (24*24) => 10 * ( 12*12 )
        x = F.relu(F.max_pool2d(self.conv1(x), 2))

        # in 20* (8*8 ) => 20* (4*4 )
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))

        # in 20* (4*4 ) => -1 * 320 
        x = x.view(-1, 320)
        # in  -1 * 320 =>  -1 * 50 
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        # in -1 * 50 => -1 * 10
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
if __name__ == '__main__':
	Net =Net()
	'''torchsummary 打印网络结构'''
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
	model = Net.to(device)     
	print(model)
	print("===>1")     
	summary(model, (1, 28, 28))#（1，28，28）对应（c,h,w）,不用指定batch参数
	print("===>2")
	'''tensorboardX生成日志文件'''
	dummy_input01 = torch.rand(10, 1, 28, 28)  # 假设输入10张1*28*28的图片
	X = dummy_input01.to(device)
	with SummaryWriter(comment='Net') as w:
	    w.add_graph(Net, (X))
	# tensorboard.exe --logdir=F:\code\CNN02\runs\Jul20_09-29-44_LAPTOP-DTN3ORVQCNN02 --host=127.0.0.1


Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)
===>1
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 24, 24]             260
            Conv2d-2             [-1, 20, 8, 8]           5,020
         Dropout2d-3             [-1, 20, 8, 8]               0
            Linear-4                   [-1, 50]          16,050
            Linear-5                   [-1, 10]             510
Total params: 21,840
Trainable params: 21,840
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.08
Estimated Total Size (MB): 0.1